# LoRA Serving

SGLang enables the use of [LoRA adapters](https://arxiv.org/abs/2106.09685) with a base model. By incorporating techniques from [S-LoRA](https://arxiv.org/pdf/2311.03285) and [Punica](https://arxiv.org/pdf/2310.18547), SGLang can efficiently support multiple LoRA adapters for different sequences within a single batch of inputs.

## Arguments for LoRA Serving

The following server arguments are relevant for multi-LoRA serving:

* `lora_paths`: A mapping from each adaptor's name to its path, in the form of `{name}={path} {name}={path}`.

* `max_loras_per_batch`: Maximum number of adaptors used by each batch. This argument can affect the amount of GPU memory reserved for multi-LoRA serving, so it should be set to a smaller value when memory is scarce. Defaults to be 8.

* `lora_backend`: The backend of running GEMM kernels for Lora modules. It can be one of `triton` or `flashinfer`, and set to `triton` by default. For better performance and stability, we recommend using the Triton LoRA backend. In the future, faster backend built upon Cutlass or Cuda kernels will be added.

* `tp_size`: LoRA serving along with Tensor Parallelism is supported by SGLang. `tp_size` controls the number of GPUs for tensor parallelism. More details on the tensor sharding strategy can be found in [S-Lora](https://arxiv.org/pdf/2311.03285) paper.

From client side, the user needs to provide a list of strings as input batch, and a list of adaptor names that each input sequence corresponds to.

## Usage

### Serving Single Adaptor

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, terminate_process

import json
import requests

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct \
    --lora-paths lora0=algoprog/fact-generation-llama-3.1-8b-instruct-lora \
    --max-loras-per-batch 1 --lora-backend triton \
    --disable-radix-cache
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-07 21:50:06] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3.1-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='127.0.0.1', port=39587, nccl_port=None, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=48483

[2025-07-07 21:50:19] Attention backend not set. Use fa3 backend by default.
[2025-07-07 21:50:19] Init torch distributed begin.


[2025-07-07 21:50:20] Init torch distributed ends. mem usage=0.00 GB


[2025-07-07 21:50:22] Load weight begin. avail mem=75.96 GB


[2025-07-07 21:50:23] Using model weights format ['*.safetensors']


...

[2025-07-07 21:50:28] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=41.86 GB


[2025-07-07 21:50:29] INFO:     Started server process [1363043]
[2025-07-07 21:50:29] INFO:     Waiting for application startup.
[2025-07-07 21:50:29] INFO:     Application startup complete.
[2025-07-07 21:50:29] INFO:     Uvicorn running on http://127.0.0.1:39587 (Press CTRL+C to quit)


[2025-07-07 21:50:30] INFO:     127.0.0.1:40180 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-07 21:50:30] INFO:     127.0.0.1:40182 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-07 21:50:30] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-07T21:50:30.382555


[2025-07-07 21:50:31] INFO:     127.0.0.1:40184 - "POST /generate HTTP/1.1" 200 OK
[2025-07-07 21:50:31] The server is fired up and ready to roll!


In [3]:
url = f"http://127.0.0.1:{port}"
json_data = {
    "text": [
        "List 3 countries and their capitals.",
        "AI is a field of computer science focused on",
    ],
    "sampling_params": {"max_new_tokens": 32, "temperature": 0},
    # The first input uses lora0, and the second input uses the base model
    "lora_path": ["lora0", None],
}
response = requests.post(
    url + "/generate",
    json=json_data,
)
print(f"Output 0: {response.json()[0]['text']}")
print(f"Output 1: {response.json()[1]['text']}")

[2025-07-07 21:50:35] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-07T21:50:35.024922


[2025-07-07 21:50:36] Decode batch. #running-req: 1, #token: 0, token usage: 0.00, cuda graph: False, gen throughput (token/s): 5.37, #queue-req: 1, timestamp: 2025-07-07T21:50:36.059862
[2025-07-07 21:50:36] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-07T21:50:36.060099


[2025-07-07 21:50:36] INFO:     127.0.0.1:52946 - "POST /generate HTTP/1.1" 200 OK
Output 0:  Each country and capital should be on a new line. 
France, Paris
Japan, Tokyo
Brazil, Brasília
List 3 countries and their capitals
Output 1:  creating intelligent machines that can perform tasks that typically require human intelligence, such as learning, problem-solving, decision-making, and perception. AI has many applications in various


In [4]:
terminate_process(server_process)

[2025-07-07 21:50:36] Child process unexpectedly failed with exitcode=9. pid=1364030


### Serving Multiple Adaptors

In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct \
    --lora-paths lora0=algoprog/fact-generation-llama-3.1-8b-instruct-lora \
    lora1=Nutanix/Meta-Llama-3.1-8B-Instruct_lora_4_alpha_16 \
    --max-loras-per-batch 2 --lora-backend triton \
    --disable-radix-cache
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-07 21:50:48] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3.1-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='127.0.0.1', port=34006, nccl_port=None, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=95315

[2025-07-07 21:51:00] Attention backend not set. Use fa3 backend by default.
[2025-07-07 21:51:00] Init torch distributed begin.


[2025-07-07 21:51:00] Init torch distributed ends. mem usage=0.00 GB


[2025-07-07 21:51:01] Load weight begin. avail mem=43.03 GB


[2025-07-07 21:51:02] Using model weights format ['*.safetensors']


...

[2025-07-07 21:51:07] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=22.12 GB


[2025-07-07 21:51:08] INFO:     Started server process [1366160]
[2025-07-07 21:51:08] INFO:     Waiting for application startup.
[2025-07-07 21:51:08] INFO:     Application startup complete.
[2025-07-07 21:51:08] INFO:     Uvicorn running on http://127.0.0.1:34006 (Press CTRL+C to quit)


[2025-07-07 21:51:09] INFO:     127.0.0.1:60022 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-07 21:51:09] INFO:     127.0.0.1:60028 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-07 21:51:09] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-07T21:51:09.591372


[2025-07-07 21:51:11] INFO:     127.0.0.1:60042 - "POST /generate HTTP/1.1" 200 OK
[2025-07-07 21:51:11] The server is fired up and ready to roll!


In [6]:
url = f"http://127.0.0.1:{port}"
json_data = {
    "text": [
        "List 3 countries and their capitals.",
        "AI is a field of computer science focused on",
    ],
    "sampling_params": {"max_new_tokens": 32, "temperature": 0},
    # The first input uses lora0, and the second input uses lora1
    "lora_path": ["lora0", "lora1"],
}
response = requests.post(
    url + "/generate",
    json=json_data,
)
print(f"Output 0: {response.json()[0]['text']}")
print(f"Output 1: {response.json()[1]['text']}")

[2025-07-07 21:51:14] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-07T21:51:14.074603
[2025-07-07 21:51:14] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 0, #token: 9, token usage: 0.00, #running-req: 1, #queue-req: 0, timestamp: 2025-07-07T21:51:14.075938


[2025-07-07 21:51:15] Decode batch. #running-req: 2, #token: 0, token usage: 0.00, cuda graph: False, gen throughput (token/s): 8.94, #queue-req: 0, timestamp: 2025-07-07T21:51:15.683323
[2025-07-07 21:51:15] INFO:     127.0.0.1:60054 - "POST /generate HTTP/1.1" 200 OK
Output 0:  Each country and capital should be on a new line. 
France, Paris
Japan, Tokyo
Brazil, Brasília
List 3 countries and their capitals
Output 1:  creating intelligent machines capable of performing tasks that typically require human intelligence. AI research has led to the development of various techniques, including machine learning, natural language processing,


In [7]:
terminate_process(server_process)

[2025-07-07 21:51:15] Child process unexpectedly failed with exitcode=9. pid=1367210


## Future Works

The development roadmap for LoRA-related features can be found in this [issue](https://github.com/sgl-project/sglang/issues/2929). Currently radix attention is incompatible with LoRA and must be manually disabled. Other features, including Unified Paging, Cutlass backend, and dynamic loading/unloadingm, are still under development.